In [3]:
import os

os.chdir('/Users/gabriele/PycharmProjects/RecSys')

from src.HyperparameterTuning.SearchAbstractClass import SearchInputRecommenderArgs
from src.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Utils.utils import create_URM,create_ICM_with_events,combine_matrices
from src.Evaluation.Evaluator import EvaluatorHoldout
from src.Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from skopt.space import Real, Integer, Categorical

##Best config is 81: {'topK': 93, 'alpha': 0.5423746227005897, 'beta': 0.5795196737709303, 'implicit': True, 'normalize_similarity': True}

In [4]:
URM = create_URM()
ICM = create_ICM_with_events()

URM_train_validation,URM_test = split_train_in_two_percentage_global_sample(URM,train_percentage=0.85)
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation,train_percentage=0.85)

combined_matrices = combine_matrices(ICM=ICM, URM=URM_train)
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])



EvaluatorHoldout: Ignoring 13641 ( 0.1%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13628 ( 0.2%) Users that have less than 1 test interactions


In [5]:
from src.HyperparameterTuning.SearchBayesianSkopt import SearchBayesianSkopt
hyperparameters_range_dictionary = {
    "topK": Integer(10,500),
    "alpha": Real(0.1,1.0),
    "beta": Real(0.1,1.0),
    "implicit": Categorical([True]),
    "normalize_similarity": Categorical([True])
}
recommender_input_args = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [combined_matrices],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)
recommender_input_args_last_test = SearchInputRecommenderArgs(
    CONSTRUCTOR_POSITIONAL_ARGS = [URM_train_validation],
    CONSTRUCTOR_KEYWORD_ARGS = {},
    FIT_POSITIONAL_ARGS = [],
    FIT_KEYWORD_ARGS = {}
)

tuning_class = SearchBayesianSkopt(recommender_class=RP3betaRecommender,
                                   evaluator_validation=evaluator_validation,
                                   evaluator_test=evaluator_test)

n_cases = 100
n_random_starts = n_cases*0.3
output_folder_path = "logs/"

tuning_class.search(recommender_input_args=recommender_input_args,
                    hyperparameter_search_space=hyperparameters_range_dictionary,
                    metric_to_optimize="MAP",
                    cutoff_to_optimize= 10,
                    n_cases= n_cases,
                    n_random_starts=n_random_starts,
                    output_folder_path=output_folder_path,
                    output_file_name_root= RP3betaRecommender.RECOMMENDER_NAME,
                    save_model="best"
                    )

Iteration No: 1 started. Evaluating function at random point.
SearchBayesianSkopt: Testing config: {'topK': 413, 'alpha': 0.7169799352274391, 'beta': 0.9987427022417503, 'implicit': True, 'normalize_similarity': True}
RP3betaRecommender: URM Detected 10245 ( 2.8%) users with no interactions.
EvaluatorHoldout: Processed 13628 (100.0%) in 16.22 sec. Users per second: 840
SearchBayesianSkopt: New best config found. Config 0: {'topK': 413, 'alpha': 0.7169799352274391, 'beta': 0.9987427022417503, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.0180804, PRECISION_RECALL_MIN_DEN: 0.0182594, RECALL: 0.0044936, MAP: 0.0061822, MAP_MIN_DEN: 0.0062732, MRR: 0.0502676, NDCG: 0.0186553, F1: 0.0071983, HIT_RATE: 0.1502788, ARHR_ALL_HITS: 0.0555961, NOVELTY: 0.0088161, AVERAGE_POPULARITY: 0.0173249, DIVERSITY_MEAN_INTER_LIST: 0.9478339, DIVERSITY_HERFINDAHL: 0.9947764, COVERAGE_ITEM: 0.3237721, COVERAGE_ITEM_CORRECT: 0.0545988, COVERAGE_USER: 0.9983883, COVERAGE_USER_CORRECT: 

EvaluatorHoldout: Processed 13628 (100.0%) in 12.36 sec. Users per second: 1103
SearchBayesianSkopt: New best config found. Config 4: {'topK': 247, 'alpha': 0.35800645245035934, 'beta': 0.34194086370147786, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.1996845, PRECISION_RECALL_MIN_DEN: 0.2021933, RECALL: 0.0544948, MAP: 0.0929421, MAP_MIN_DEN: 0.0939271, MRR: 0.4141565, NDCG: 0.2071062, F1: 0.0856227, HIT_RATE: 0.8649839, ARHR_ALL_HITS: 0.6185072, NOVELTY: 0.0055470, AVERAGE_POPULARITY: 0.5272757, DIVERSITY_MEAN_INTER_LIST: 0.8121862, DIVERSITY_HERFINDAHL: 0.9812127, COVERAGE_ITEM: 0.0848884, COVERAGE_ITEM_CORRECT: 0.0421950, COVERAGE_USER: 0.9983883, COVERAGE_USER_CORRECT: 0.8635897, DIVERSITY_GINI: 0.0066714, SHANNON_ENTROPY: 6.8441105, RATIO_DIVERSITY_HERFINDAHL: 0.9815703, RATIO_DIVERSITY_GINI: 0.0249214, RATIO_SHANNON_ENTROPY: 0.5479250, RATIO_AVERAGE_POPULARITY: 3.3187704, RATIO_NOVELTY: 0.7041250, 

EvaluatorHoldout: Processed 13641 (100.0%) in 13.03 s

EvaluatorHoldout: Processed 13628 (100.0%) in 11.37 sec. Users per second: 1199
SearchBayesianSkopt: Config 9 is suboptimal. Config: {'topK': 220, 'alpha': 0.5376960943782237, 'beta': 0.14257194991540756, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.1662240, PRECISION_RECALL_MIN_DEN: 0.1681924, RECALL: 0.0447457, MAP: 0.0769554, MAP_MIN_DEN: 0.0777387, MRR: 0.3769761, NDCG: 0.1761605, F1: 0.0705106, HIT_RATE: 0.7977693, ARHR_ALL_HITS: 0.5363729, NOVELTY: 0.0062596, AVERAGE_POPULARITY: 0.2292103, DIVERSITY_MEAN_INTER_LIST: 0.9654041, DIVERSITY_HERFINDAHL: 0.9965333, COVERAGE_ITEM: 0.1441387, COVERAGE_ITEM_CORRECT: 0.0673348, COVERAGE_USER: 0.9983883, COVERAGE_USER_CORRECT: 0.7964835, DIVERSITY_GINI: 0.0229884, SHANNON_ENTROPY: 9.0200166, RATIO_DIVERSITY_HERFINDAHL: 0.9968965, RATIO_DIVERSITY_GINI: 0.0858746, RATIO_SHANNON_ENTROPY: 0.7221234, RATIO_AVERAGE_POPULARITY: 1.4426918, RATIO_NOVELTY: 0.7945766, 

Iteration No: 10 ended. Evaluation done at random point

EvaluatorHoldout: Processed 13628 (100.0%) in 12.30 sec. Users per second: 1108
SearchBayesianSkopt: New best config found. Config 15: {'topK': 205, 'alpha': 0.45310084023569197, 'beta': 0.3690371602566085, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.2016877, PRECISION_RECALL_MIN_DEN: 0.2043017, RECALL: 0.0554395, MAP: 0.0937505, MAP_MIN_DEN: 0.0948049, MRR: 0.4162313, NDCG: 0.2088940, F1: 0.0869722, HIT_RATE: 0.8682125, ARHR_ALL_HITS: 0.6228988, NOVELTY: 0.0056042, AVERAGE_POPULARITY: 0.5082623, DIVERSITY_MEAN_INTER_LIST: 0.8299299, DIVERSITY_HERFINDAHL: 0.9829869, COVERAGE_ITEM: 0.1005039, COVERAGE_ITEM_CORRECT: 0.0486738, COVERAGE_USER: 0.9983883, COVERAGE_USER_CORRECT: 0.8668132, DIVERSITY_GINI: 0.0083474, SHANNON_ENTROPY: 7.1143631, RATIO_DIVERSITY_HERFINDAHL: 0.9833452, RATIO_DIVERSITY_GINI: 0.0311824, RATIO_SHANNON_ENTROPY: 0.5695608, RATIO_AVERAGE_POPULARITY: 3.1990970, RATIO_NOVELTY: 0.7113835, 

EvaluatorHoldout: Processed 13641 (100.0%) in 12.19 s

EvaluatorHoldout: Processed 13628 (100.0%) in 12.93 sec. Users per second: 1054
SearchBayesianSkopt: Config 21 is suboptimal. Config: {'topK': 232, 'alpha': 0.6329556997117096, 'beta': 0.4390226358124346, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.1983196, PRECISION_RECALL_MIN_DEN: 0.2007265, RECALL: 0.0539101, MAP: 0.0926562, MAP_MIN_DEN: 0.0935660, MRR: 0.4131944, NDCG: 0.2060900, F1: 0.0847753, HIT_RATE: 0.8622689, ARHR_ALL_HITS: 0.6168055, NOVELTY: 0.0055755, AVERAGE_POPULARITY: 0.5204660, DIVERSITY_MEAN_INTER_LIST: 0.8187620, DIVERSITY_HERFINDAHL: 0.9818702, COVERAGE_ITEM: 0.1089761, COVERAGE_ITEM_CORRECT: 0.0437455, COVERAGE_USER: 0.9983883, COVERAGE_USER_CORRECT: 0.8608791, DIVERSITY_GINI: 0.0075833, SHANNON_ENTROPY: 6.9412670, RATIO_DIVERSITY_HERFINDAHL: 0.9822281, RATIO_DIVERSITY_GINI: 0.0283278, RATIO_SHANNON_ENTROPY: 0.5557031, RATIO_AVERAGE_POPULARITY: 3.2759094, RATIO_NOVELTY: 0.7077468, 

Iteration No: 22 ended. Evaluation done at random point

...

EvaluatorHoldout: Processed 13628 (100.0%) in 17.53 sec. Users per second: 777
SearchBayesianSkopt: Config 75 is suboptimal. Config: {'topK': 500, 'alpha': 1.0, 'beta': 0.8077536403163263, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.1733857, PRECISION_RECALL_MIN_DEN: 0.1743802, RECALL: 0.0432161, MAP: 0.0755438, MAP_MIN_DEN: 0.0757675, MRR: 0.3293420, NDCG: 0.1723894, F1: 0.0691874, HIT_RATE: 0.7753155, ARHR_ALL_HITS: 0.4987462, NOVELTY: 0.0062861, AVERAGE_POPULARITY: 0.4179998, DIVERSITY_MEAN_INTER_LIST: 0.9240525, DIVERSITY_HERFINDAHL: 0.9923985, COVERAGE_ITEM: 0.2817985, COVERAGE_ITEM_CORRECT: 0.0413090, COVERAGE_USER: 0.9983883, COVERAGE_USER_CORRECT: 0.7740659, DIVERSITY_GINI: 0.0324419, SHANNON_ENTROPY: 8.5961282, RATIO_DIVERSITY_HERFINDAHL: 0.9927602, RATIO_DIVERSITY_GINI: 0.1211886, RATIO_SHANNON_ENTROPY: 0.6881878, RATIO_AVERAGE_POPULARITY: 2.6309680, RATIO_NOVELTY: 0.7979375, 

Iteration No: 76 ended. Search finished for the next optimal point.
Tim

EvaluatorHoldout: Processed 13628 (100.0%) in 12.29 sec. Users per second: 1109
SearchBayesianSkopt: New best config found. Config 81: {'topK': 93, 'alpha': 0.5423746227005897, 'beta': 0.5795196737709303, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.2011667, PRECISION_RECALL_MIN_DEN: 0.2036898, RECALL: 0.0551613, MAP: 0.0948419, MAP_MIN_DEN: 0.0960226, MRR: 0.4218398, NDCG: 0.2100540, F1: 0.0865814, HIT_RATE: 0.8683593, ARHR_ALL_HITS: 0.6302041, NOVELTY: 0.0057376, AVERAGE_POPULARITY: 0.4714392, DIVERSITY_MEAN_INTER_LIST: 0.8679856, DIVERSITY_HERFINDAHL: 0.9867922, COVERAGE_ITEM: 0.1561548, COVERAGE_ITEM_CORRECT: 0.0694944, COVERAGE_USER: 0.9983883, COVERAGE_USER_CORRECT: 0.8669597, DIVERSITY_GINI: 0.0145969, SHANNON_ENTROPY: 7.7278710, RATIO_DIVERSITY_HERFINDAHL: 0.9871518, RATIO_DIVERSITY_GINI: 0.0545276, RATIO_SHANNON_ENTROPY: 0.6186770, RATIO_AVERAGE_POPULARITY: 2.9673255, RATIO_NOVELTY: 0.7283164, 

EvaluatorHoldout: Processed 13641 (100.0%) in 12.20 sec

Iteration No: 87 ended. Search finished for the next optimal point.
Time taken: 34.2200
Function value obtained: -0.0775
Current minimum: -0.0948
Iteration No: 88 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 272, 'alpha': 0.7768860542319304, 'beta': 0.5912170553255992, 'implicit': True, 'normalize_similarity': True}
RP3betaRecommender: URM Detected 10245 ( 2.8%) users with no interactions.
EvaluatorHoldout: Processed 13628 (100.0%) in 14.06 sec. Users per second: 970
SearchBayesianSkopt: Config 87 is suboptimal. Config: {'topK': 272, 'alpha': 0.7768860542319304, 'beta': 0.5912170553255992, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.1971969, PRECISION_RECALL_MIN_DEN: 0.1994242, RECALL: 0.0530145, MAP: 0.0924952, MAP_MIN_DEN: 0.0933647, MRR: 0.4130015, NDCG: 0.2052645, F1: 0.0835637, HIT_RATE: 0.8567655, ARHR_ALL_HITS: 0.6157678, NOVELTY: 0.0055938, AVERAGE_POPULARITY: 0.5209317, DIVERSITY_MEAN_INTER_LIST: 0.834

Iteration No: 93 ended. Search finished for the next optimal point.
Time taken: 35.5266
Function value obtained: -0.0856
Current minimum: -0.0948
Iteration No: 94 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 445, 'alpha': 0.9977889553705231, 'beta': 0.499647627099825, 'implicit': True, 'normalize_similarity': True}
RP3betaRecommender: URM Detected 10245 ( 2.8%) users with no interactions.
EvaluatorHoldout: Processed 13628 (100.0%) in 14.87 sec. Users per second: 917
SearchBayesianSkopt: Config 93 is suboptimal. Config: {'topK': 445, 'alpha': 0.9977889553705231, 'beta': 0.499647627099825, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.1945700, PRECISION_RECALL_MIN_DEN: 0.1964656, RECALL: 0.0514660, MAP: 0.0903401, MAP_MIN_DEN: 0.0909201, MRR: 0.4000819, NDCG: 0.2009168, F1: 0.0814006, HIT_RATE: 0.8447314, ARHR_ALL_HITS: 0.5996767, NOVELTY: 0.0056255, AVERAGE_POPULARITY: 0.5315838, DIVERSITY_MEAN_INTER_LIST: 0.82088

Iteration No: 99 ended. Search finished for the next optimal point.
Time taken: 39.6613
Function value obtained: -0.0922
Current minimum: -0.0948
Iteration No: 100 started. Searching for the next optimal point.
SearchBayesianSkopt: Testing config: {'topK': 75, 'alpha': 0.23255973953938447, 'beta': 0.2759955870354426, 'implicit': True, 'normalize_similarity': True}
RP3betaRecommender: URM Detected 10245 ( 2.8%) users with no interactions.
EvaluatorHoldout: Processed 13628 (100.0%) in 10.88 sec. Users per second: 1253
SearchBayesianSkopt: Config 99 is suboptimal. Config: {'topK': 75, 'alpha': 0.23255973953938447, 'beta': 0.2759955870354426, 'implicit': True, 'normalize_similarity': True} - results: PRECISION: 0.1901967, PRECISION_RECALL_MIN_DEN: 0.1926253, RECALL: 0.0527725, MAP: 0.0889117, MAP_MIN_DEN: 0.0900673, MRR: 0.4118588, NDCG: 0.1999524, F1: 0.0826208, HIT_RATE: 0.8505283, ARHR_ALL_HITS: 0.6031329, NOVELTY: 0.0061058, AVERAGE_POPULARITY: 0.3201271, DIVERSITY_MEAN_INTER_LIST: 0.9